# Historical "high school activities" data on Earlham College applicants (2018-'21)

- This script inputs four Excel sheets containing data on all Earlham College applicants from 2018-2021.  
- Each file has somewhere between 900-1500 rows, with each row having 56 columns of data - 40 of which contain information that students wrote on their applications regarding the names and types of clubs and extracurricular activities they did in high school.
- The end result is a new Excel file called 'Total Clubs 2018-2021' which is an ordered list of the most common clubs and activities that Earlham applicants were a part of, with their counts for each year (2018, 19, 20, 21) and a "Total Counts" column.

In [30]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import xlsxwriter

# Output file name

output = 'Total Clubs 2018-2021.xlsx'

# Current directory

directory = os.getcwd()

# IF output file already exists, delete it

if os.path.exists(output):
            os.remove(output)
        
# Create list titled 'Files' which contains the path for each of my four Excel sheets        

files = [filename for filename in os.listdir(directory) if filename.startswith("App")]

# Create new Df, and then turn it into a list, containing all of the HS clubs that we want to check, then making
# everything in 'mylist' lowercase

df2 = pd.read_excel('club_list.xlsx')
list_of_clubs = df2['Clubs'].tolist()
list_of_clubs = [x.lower() for x in list_of_clubs]

for i in files:
    
    # Inputting my Excel sheet(s) into a DF
    
    app = pd.read_excel(i)
    
    # Replacing ", / - ;" with white spaces to clean up data
    
    app = app.replace(',',' ', regex=True).replace('/',' ', regex=True).replace('-',' ',regex=True).replace(';',' ',regex=True)
    
    # Fill null values with white space
    
    app.fillna(' ', inplace = True)
    
    # Taking all of the string data from 'Interest Name', 'Interest Description', and 'Interest Role' columns and
    # combining them into Activity0, Activity1, Activity2, etc etc all the way to Activity 9
    
    ct = 0
    while ct <= 9:
        if ct == 0:
            app['Activity'+str(ct)] = app['Interest Name'].astype(str) + ' ' + app['Interest Description'].astype(str) + ' ' + app['Interest Role'].astype(str)
        else:
            app['Activity'+str(ct)] = app['Interest Name' + '.' + str(ct)].astype(str) + ' ' + app['Interest Description' + '.' + str(ct)].astype(str) + ' ' + app['Interest Role' + '.' + str(ct)].astype(str)
        ct += 1
    
    # For each student, combining their nine "Activity" columns into  single column titled 'Activities'
    
    app['Activities'] = app['Activity0'] + app['Activity1'] + app['Activity2'] + app['Activity3'] + app['Activity4'] + app['Activity5'] + app['Activity6'] + app['Activity7'] + app['Activity8'] + app['Activity9']
        
    # Removing any leading or trailing white spaces from Activities column (cleaning data)
    
    for col in app[['Activities']]:
        app[col] = app[col].str.strip()
    
    # Creating a list out of my activities column and then making everything lowercase
    
    unique_cols = app['Activities'].unique().tolist()
    unique_cols = [x.lower() for x in unique_cols]
    
    # For each of the following: Counting the number of occurences for every club in mylist if it shows up in
    # my Excel file.  Appending to new list named after the year the Excel file references
    
    if '2018' in i:
        _2018 = [l for l in list_of_clubs for c in unique_cols if l in c]

    elif '2019' in i:
        _2019 = [l for l in list_of_clubs for c in unique_cols if l in c]
                    
    elif '2020' in i:
        _2020 = [l for l in list_of_clubs for c in unique_cols if l in c]
                    
    elif '2021' in i:
        _2021 = [l for l in list_of_clubs for c in unique_cols if l in c]
        
# Adding up occurences from each of the four lists into a new 'total' list so I can add a 'Total' column to Excel

total = _2018 + _2019 + _2020 + _2021

# Creating a new dictionary to store my values.  Keys = club names and values = # of times that club was listed on an application

clubs_and_counts = {}

# Adding each year and then the total count to my dictionary as values.  Keys are the club names

for l in mylist:
    clubs_and_counts[l] = [_2021.count(l), _2020.count(l), _2019.count(l), _2018.count(l), total.count(l)]

# Getting rid of dictinoary entries where 'Total' == 0

for k in clubs_and_counts.copy():
    if clubs_and_counts.get(k)[-1] == 0:
        clubs_and_counts.pop(k)
        
# Creating a new DF from my dictionary

sorted_clubs = pd.DataFrame.from_dict(clubs_and_counts, orient='index')

# Labeling my columns

sorted_clubs.columns = ['2021', '2020','2019', '2018', 'Total']

# Creating my writer variable

writer = pd.ExcelWriter(output)

# Using pd.to_Excel to send my DF to Excel.  Sorting my values by 'Total' in reverse ascending order.  Naming
# my sheet 'EC Applicants Clubs_Activities' and my index 'Club Name'

sorted_clubs.sort_values(by='Total', ascending=False).to_excel(writer, sheet_name = 'EC Applicants Clubs_Activities', index_label = "Club Name")

# Manually setting column A to be a bit wider

worksheet = writer.sheets['EC Applicants Clubs_Activities']
worksheet.set_column('A:A', 25)

# Setting column F ('Total') to bold and freezing the top row and first column

workbook = xlsxwriter.Workbook(output)
cell_format = workbook.add_format()
cell_format.set_bold()
worksheet.set_column('F:F', None, cell_format)
worksheet.freeze_panes(1,1)

#Saving!

writer.save()
print('All done :-)')

All done :-)
